In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import os
import sys

base_dir = "/content/gdrive/MyDrive/Workspace/Colab"
sys.path.append(base_dir)

src_dir = os.path.join(base_dir, "langchain")
%cd $src_dir

/content/gdrive/MyDrive/Workspace/Colab/langchain


In [3]:
# error log만 출력
!pip install -r requirements.txt > /dev/null

### **Configure Redis On Colab**

* [Running Redis On Google Colab](https://redis.io/blog/running-redis-on-google-colab/)
* [Redis-stack Release](https://github.com/redis-stack/redis-stack/releases)
* [Openssl, expired Check](https://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/?C=M;O=D)

In [4]:
!mkdir -p /content/volume

!curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-7.2.0-v13.focal.x86_64.tar.gz -o /content/volume/redis-stack-server.tar.gz
!tar -xvf /content/volume/redis-stack-server.tar.gz -C /content/volume/

./
./redis-stack-server-7.2.0-v13/
./redis-stack-server-7.2.0-v13/share/
./redis-stack-server-7.2.0-v13/share/APACHE_LICENSE
./redis-stack-server-7.2.0-v13/share/RSALv2.txt
./redis-stack-server-7.2.0-v13/share/SSPLv1.txt
./redis-stack-server-7.2.0-v13/lib/
./redis-stack-server-7.2.0-v13/lib/rejson.so
./redis-stack-server-7.2.0-v13/lib/redistimeseries.so
./redis-stack-server-7.2.0-v13/lib/libredisgears_v8_plugin.so
./redis-stack-server-7.2.0-v13/lib/redisearch.so
./redis-stack-server-7.2.0-v13/lib/redisbloom.so
./redis-stack-server-7.2.0-v13/lib/rediscompat.so
./redis-stack-server-7.2.0-v13/lib/redisgears.so
./redis-stack-server-7.2.0-v13/etc/
./redis-stack-server-7.2.0-v13/etc/redis-stack.conf
./redis-stack-server-7.2.0-v13/etc/redis-stack-service.conf
./redis-stack-server-7.2.0-v13/etc/README
./redis-stack-server-7.2.0-v13/bin/
./redis-stack-server-7.2.0-v13/bin/redis-cli
./redis-stack-server-7.2.0-v13/bin/redis-check-rdb
./redis-stack-server-7.2.0-v13/bin/redis-benchmark
./redis-stac

In [5]:
!curl -fsSL http://nz2.archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2.23_amd64.deb -o /content/volume/libssl1.1_1.1.1f-1ubuntu2.23_amd64.deb
!sudo dpkg -i /content/volume/libssl1.1_1.1.1f-1ubuntu2.23_amd64.deb

Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 123632 files and directories currently installed.)
Preparing to unpack .../libssl1.1_1.1.1f-1ubuntu2.23_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1f-1ubuntu2.23) ...
Setting up libssl1.1:amd64 (1.1.1f-1ubuntu2.23) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ld

In [11]:
# Run Redis-stack-server (Background)
!/content/volume/redis-stack-server-7.2.0-v13/bin/redis-stack-server --daemonize yes

Starting redis-stack-server, database path /content/volume/redis-stack-server-7.2.0-v13/var/db/redis-stack


In [12]:
import redis
client = redis.Redis(host = 'localhost', port=6379)
client.ping()

True

## **Model**

In [13]:
import import_ipynb
from utils.decorator import *

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")
REDIS_URL = os.getenv("REDIS_URL")

if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found in .env file.")
if not SERP_API_KEY:
    raise ValueError("Goodle Search API key not found in .env file.")
if not REDIS_URL:
    raise ValueError("Redis URL not found in .env file.")

In [15]:
from langchain_community.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_llm_cache, set_debug
from langchain_community.cache import RedisCache, InMemoryCache, RedisSemanticCache
from langchain.embeddings import OpenAIEmbeddings
from redis import Redis

In [16]:
# cache = RedisSemanticCache(
#     redis_url=REDIS_URL,
#     embedding=OpenAIEmbeddings()
# )

try:
    # Upstash 사용으로 변경
    cache = RedisCache(

      redis_= Redis(
          host="localhost",
          port=6379
      )
    )

    cache.clear()
    set_debug(False)
    set_llm_cache(cache)

except Exception as e:
    print(e)

* temperature: 모델 출력의 창의성, 무작위성 조정 (확률분포의 분산 조절)
* top_p: 누적확률샘플링 기반, 모델이 단어를 선택할 확률 범위 제한

In [17]:
class TestModel:

    def __init__(self):
        self.model = OpenAI(
            api_key=OPENAI_API_KEY,
            temperature=0.0,
            top_p=1.0,
            n=2,
            best_of=2,
            cache=True
            )

    @elapsed_time
    def get_response_sync(self, prompt: str) -> str:
        return self._invoke_model(prompt)

    @elapsed_time
    async def get_response_async(self, prompt: str) -> str:
        return await self._invoke_model_async(prompt)

    def _invoke_model(self, prompt: str) -> str:
        try:
            return self.model.invoke(prompt)
        except Exception as e:
            raise ValueError(f"모델 호출 중 오류 발생: {str(e)}")

    async def _invoke_model_async(self, prompt: str) -> str:
        try:
            return await self.model.ainvoke(prompt)
        except Exception as e:
            raise ValueError(f"비동기 모델 호출 중 오류 발생: {str(e)}")

In [18]:
import json


def _get_meta_info(callback) -> dict:

    return {
        "Tokens Used": callback.total_tokens,
        "Prompt Tokens": callback.prompt_tokens,
        "Completion Tokens": callback.completion_tokens,
        "Total Cost (USD)": callback.total_cost,
        "Successful Requests": callback.successful_requests,
    }


def _convert_to_pretty_json(data: dict) -> str:
    return json.dumps(data, indent=4)

In [19]:
llm = TestModel()

<ipython-input-17-117291d638e2>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  self.model = OpenAI(


In [20]:
prompt = "What is famous street foods in Seoul Korea in 200 characters"

with get_openai_callback() as cb:

    response = llm.get_response_sync(prompt)
    print(_convert_to_pretty_json(_get_meta_info(cb)))

# Caching
with get_openai_callback() as cb:

    response = llm.get_response_sync(prompt)
    print(_convert_to_pretty_json(_get_meta_info(cb)))

Sync execution time: 1.430206834000046
{
    "Tokens Used": 236,
    "Prompt Tokens": 12,
    "Completion Tokens": 224,
    "Total Cost (USD)": 0.000466,
    "Successful Requests": 1
}
Sync execution time: 0.0013854999999693973
{
    "Tokens Used": 0,
    "Prompt Tokens": 0,
    "Completion Tokens": 0,
    "Total Cost (USD)": 0.0,
    "Successful Requests": 0
}


In [21]:
# 모든 키, 검색
keys = client.keys('*')

for key in keys:

    key_type = client.type(key)

    if key_type == b'string':
        value = client.get(key)
        print(f"Key: {key.decode()}, Value: {value.decode()}")
    elif key_type == b'hash':
        value = client.hgetall(key)
        print(f"Key: {key.decode()} is a hash, Fields and Values:")

        for field, val in value.items():
            field_decoded = field.decode()
            val_decoded = val.decode()
            print(f"  Field: {field_decoded}, Value: {val_decoded}")

        print()
    else:
        print(f"Key: {key.decode()} is not a string or hash, it is a {key_type.decode()}.")

Key: 0715e99ff5dd15cabf02492f9e22cfcd is a hash, Fields and Values:
  Field: 0, Value: {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output", "Generation"], "kwargs": {"text": "\n\n1. Tteokbokki (spicy rice cakes)\n2. Kimbap (seaweed rice rolls)\n3. Hotteok (sweet pancakes)\n4. Japchae (stir-fried glass noodles)\n5. Mandu (dumplings)\n6. Odeng (fish cake skewers)\n7. Gyeran-ppang (egg bread)\n8. Sundae (blood sausage)\n9. Twigim (fried snacks)\n10. Bungeoppang (fish-shaped pastry with red bean filling)", "generation_info": {"finish_reason": "stop", "logprobs": null}, "type": "Generation"}}
  Field: 1, Value: {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "output", "Generation"], "kwargs": {"text": "\n\n1. Tteokbokki (spicy rice cakes)\n2. Kimbap (seaweed rice rolls)\n3. Hotteok (sweet pancakes)\n4. Japchae (stir-fried glass noodles)\n5. Mandu (dumplings)\n6. Odeng (fish cake skewers)\n7. Gyeran-ppang (egg bread)\n8. Sundae (blood sausage)\n9. Twigim (f

In [22]:
import tiktoken


def _get_tokens(text: str, model_name: str = "gpt-4") -> int:
    """
    tokenizer
    """
    encoding = tiktoken.encoding_for_model(model_name)
    return encoding.encode(text)

In [23]:
tokens = _get_tokens(prompt, model_name="gpt-3.5-turbo")

print(f"Token count: {len(tokens)}")
print(f"Tokens: {tokens}")

Token count: 12
Tokens: [3923, 374, 11495, 8761, 15657, 304, 51289, 12126, 304, 220, 1049, 5885]


*Tracking*

In [24]:
from langchain.agents import load_tools, initialize_agent, AgentType


tools = load_tools(["serpapi"], serpapi_api_key=SERP_API_KEY)
agent = initialize_agent(
    tools,
    llm.model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

with get_openai_callback() as cb:

    response = agent.run(
        prompt
    )
    print(_convert_to_pretty_json(_get_meta_info(cb)))

print()

with get_openai_callback() as cb:

    response = agent.run(
        prompt
    )
    print(_convert_to_pretty_json(_get_meta_info(cb)))

<ipython-input-24-9cb5f974d499>:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(
<ipython-input-24-9cb5f974d499>:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(




> Entering new AgentExecutor chain...
 I should think about what to do
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should think about what to do
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should think about what to do
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I now know the final answer
Final Answer: According to my search, some famous street foods in Seoul Korea include tteokbokki, kimbap, and hotteok.

> Finished chain.
{
    "Tokens Used": 1106,
    "Prompt Tokens": 884,
    "Completion Tokens": 222,
    "Total Cost (USD)": 0.00177,
    "Successful Requests": 4
}



> Entering new AgentExecutor chain...
 I should think about what to do
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should think a